In [ ]:
import pandas as pd
import re

def standardize_title(title, remove_pdf=True):
    """
    Standardize title format by:
    1. Removing special characters
    2. Replacing spaces with underscores
    3. Optionally removing .pdf extension
    """
    # Remove special characters and replace spaces with underscores
    standardized = re.sub(r'[\\/*?:"<>|]', "", title).replace(" ", "_")
    
    # Remove .pdf extension if present and requested
    if remove_pdf and standardized.endswith('.pdf'):
        standardized = standardized[:-4]
        
    return standardized

def convert_to_space_format(title):
    """
    Convert underscore title to space format and remove .pdf
    """
    # Remove .pdf if present
    if title.endswith('.pdf'):
        title = title[:-4]
    # Replace underscores with spaces
    return title.replace("_", " ")

def merge_excel_files(file1_path, file2_path, output_path):
    """
    Merge two Excel files based on nos_title column with format matching
    """
    # Read both Excel files
    df1 = pd.read_excel(file1_path)
    df2 = pd.read_excel(file2_path)
    
    # Create standardized title columns for matching
    df1['standardized_title'] = df1['nos_title'].apply(lambda x: standardize_title(x, remove_pdf=False))
    df2['standardized_title'] = df2['nos_title'].apply(lambda x: standardize_title(x, remove_pdf=True))
    
    # Store the space-formatted titles from df1
    title_mapping = dict(zip(df1['standardized_title'], df1['nos_title']))
    
    # For any titles only in df2, convert them to space format
    df2_only_titles = set(df2['standardized_title']) - set(df1['standardized_title'])
    for title in df2_only_titles:
        original_title = df2.loc[df2['standardized_title'] == title, 'nos_title'].iloc[0]
        title_mapping[title] = convert_to_space_format(original_title)
    
    # Merge dataframes on standardized title
    merged_df = pd.merge(
        df1, 
        df2,
        left_on='standardized_title',
        right_on='standardized_title',
        how='outer',
        suffixes=('_1', '_2')
    )
    
    # Create single nos_title column using the mapping
    merged_df['nos_title'] = merged_df['standardized_title'].map(title_mapping)
    
    # Drop temporary and duplicate columns
    columns_to_drop = ['standardized_title', 'nos_title_1', 'nos_title_2']
    merged_df = merged_df.drop([col for col in columns_to_drop if col in merged_df.columns], axis=1)
    
    # Save to new Excel file
    merged_df.to_excel(output_path, index=False)
    
    # Print merge statistics
    print(f"File 1 rows: {len(df1)}")
    print(f"File 2 rows: {len(df2)}")
    print(f"Merged rows: {len(merged_df)}")
    
    return merged_df

# Usage example:
# file1_path = "path_to_first_excel.xlsx"  # File with "Abrasive wheel operations in the wood industry" format
# file2_path = "path_to_second_excel.xlsx" # File with "Abrasive_wheel_operations_in_the_wood_industry.pdf" format
# output_path = "path_to_output.xlsx"
# merged_df = merge_excel_files(file1_path, file2_path, output_path)